In [1]:
import cPickle
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1234)

In [2]:
with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/coco_train_context.pkl', 'r') as f:
    titles = cPickle.load(f)
    descriptions = cPickle.load(f)
    tags = cPickle.load(f)

with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/coco_align.train.pkl', 'rb') as f:
    captions = cPickle.load(f)

with open('/media/Data/flipvanrijn/datasets/coco/images/train2014list.txt', 'r') as f:
    images = f.read().split()

In [3]:
im2idx = {}
for i, data in enumerate(captions):
    cap, im = data
    if im in im2idx:
        im2idx[im].append(i)
    else:
        im2idx[im] = [i]

In [4]:
import gensim
model = gensim.models.Word2Vec.load_word2vec_format('/media/Data/flipvanrijn/models/word2vec/enwiki-latest-pages.512.bin', binary=True)

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K2200


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/tfidf_model.pkl') as f_in:
    tfidf_model = cPickle.load(f_in)
tfidf_feature_names = tfidf_model.get_feature_names()

In [39]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import wordpunct_tokenize, sent_tokenize

stop = stopwords.words('english')
stemmer = SnowballStemmer('english')

def preprocess(s, model):
    global stop, stemmer
    
    ns = []
    for w in s:
        w = stemmer.stem(w)
        if w:
            ns.append(w)
        
    return ns

def get_document(title, desc, tags):
    context = []
    title = preprocess(title.split(' '), model)
    if title:
        context += title
    # ... description (for each sentence) ...
    for sent in sent_tokenize(desc):
        sent = preprocess(sent.split(' '), model)
        if desc:
            context += sent
    # ... and tagsc
    ts = preprocess(tags, model)
    if ts:
        context += ts
    
    return ' '.join(context)

idx = 40
doc = get_document(titles[idx], descriptions[idx], tags[idx])
feats = tfidf_model.transform([doc]).todense()[0].tolist()[0]
scores = sorted([(tfidf_feature_names[pair[0]], pair[1]) for pair in zip(range(0, len(feats)), feats) if pair[1] > 0], key=lambda x: x[1] * -1)
print scores
'noodle' in model

[(u'beansprout', 0.29767941588364305), (u'civilis', 0.29767941588364305), (u'food noodl', 0.2927815452498042), (u'ricenoodl', 0.2927815452498042), (u'singapor food', 0.2927815452498042), (u'tasti food', 0.2927815452498042), (u'springrol', 0.27905876959095355), (u'carrot lettuc', 0.2741608989571147), (u'noodl', 0.21099375611502338), (u'lettuc', 0.20440880746711915), (u'food', 0.1962263782009292), (u'tasti', 0.19565741985450608), (u'asian', 0.1891616591684372), (u'singapor', 0.18715731933526472), (u'bowl', 0.15961712112906215), (u'museum', 0.15675210481196644), (u'carrot', 0.14995995793557892), (u'build', 0.14288816270076565)]


True

In [415]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop = stopwords.words('english')
stemmer = WordNetLemmatizer()

def preprocess(s, model):
    global stop, stemmer
    
    ns = []
    for w in s:
        w = stemmer.lemmatize(w)
        if w in model and w not in stop:
            ns.append(w)
        
    return ns

def sim(s1, s2, model):
    '''
    m = np.zeros((len(s1), len(s2)))
    for ii in xrange(len(s1)):
        for jj in xrange(len(s2)):
            if s1[ii] in normalized_vocab and s2[jj] in normalized_vocab:
                s1i = normalized_vocab[s1[ii]]
                s2j = normalized_vocab[s2[jj]]
                m[ii][jj] = model.similarity(s1i, s2j)
    return (m.sum() / np.count_nonzero(m))
    '''
    s1 = preprocess(s1, model)
    s2 = preprocess(s2, model)
    similarity = model.n_similarity(s1, s2)

    return np.sum(similarity)

def n_relevant(caption, title, description, tags, model, n=10):
    scores = []
    w_caption = caption.split(' ')
    w_title   = title.split(' ')
    if title:
        try:
            scores.append((sim(w_caption, w_title, model), title))
        except:
            print w_title
            raise
    for desc in description.split('.'):
        w_desc = desc.split(' ')
        if desc:
            scores.append((sim(w_caption, w_desc, model), desc))
    if tags:
        scores.append((sim(w_caption, tags, model), ' '.join(tags)))
    
    df = pd.DataFrame(scores, columns=['score', 'sentence'])
    df = df.dropna()
    df = df.sort(['score'], ascending=False)[:n]
    return df['score']

def n_relevant_words(caption, title, description, tags, model, n=100):
    def relevant(s1, s2, model):
        m = np.zeros((len(s1), len(s2)))
        for ii in xrange(len(s1)):
            for jj in xrange(len(s2)):
                if s1[ii] in model and s2[jj] in model:
                    m[ii][jj] = model.similarity(s1[ii], s2[jj])
        scores = []
        for r, c in zip(np.argmax(m, axis=0), range(len(s2))):
            scores.append((m[r][c], s2[c]))
        return scores
    
    cap = preprocess(caption, model)
    title = preprocess(title, model)
    descs = description.split('.')
    ts = preprocess(tags, model)

    scores = []
    scores += relevant(cap, title, model)
    for desc in descs:
        desc = preprocess(desc.split(' '), model)
        scores += relevant(cap, desc, model)
    scores += relevant(cap, ts, model)

    df = pd.DataFrame(scores, columns=['score', 'word'])
    df = df.sort(['score'], ascending=False)
    df = df.dropna()
    df = df.drop_duplicates()
    return df['word'][:n]

In [385]:
np.random.seed(1234)
lengths = []
for i in range(len(titles)):
    caption_idx = np.random.choice(im2idx[i])
    cap = captions[caption_idx][0].split(' ')
    title = titles[i].split(' ')
    desc = descriptions[i]
    ts = tags[i]

    relevant_words = n_relevant_words(cap, title, desc, ts, model)
    lengths.append(relevant_words.shape[0])
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000


In [435]:
from scipy.sparse import csr_matrix, vstack

images_with_context = []
for i, im in enumerate(images):
    caption_idx = np.random.choice(im2idx[i])
    cap = captions[caption_idx][0].split(' ')
    title = titles[i].split(' ')
    desc = descriptions[i]
    ts = tags[i]

    relevant_words = n_relevant_words(cap, title, desc, ts, model)
    
    if len(relevant_words) > 0:
        images_with_context.append(im)
        
        context = np.zeros((100, 300), dtype=np.float32)
        for wi, w in enumerate(relevant_words.values):
            context[wi,:] = model[w]

        if i == 0:
            feat_flatten = csr_matrix(context.flatten())
        else:
            feat_flatten = vstack([feat_flatten, csr_matrix(context.flatten())])
    
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


KeyboardInterrupt: 

(<99x30000 sparse matrix of type '<type 'numpy.float32'>'
 	with 561600 stored elements in Compressed Sparse Row format>,
 ['COCO_train2014_000000000009.jpg',
  'COCO_train2014_000000000025.jpg',
  'COCO_train2014_000000000030.jpg',
  'COCO_train2014_000000000034.jpg',
  'COCO_train2014_000000000036.jpg',
  'COCO_train2014_000000000049.jpg',
  'COCO_train2014_000000000061.jpg',
  'COCO_train2014_000000000064.jpg',
  'COCO_train2014_000000000071.jpg',
  'COCO_train2014_000000000072.jpg',
  'COCO_train2014_000000000077.jpg',
  'COCO_train2014_000000000078.jpg',
  'COCO_train2014_000000000081.jpg',
  'COCO_train2014_000000000086.jpg',
  'COCO_train2014_000000000089.jpg',
  'COCO_train2014_000000000092.jpg',
  'COCO_train2014_000000000094.jpg',
  'COCO_train2014_000000000109.jpg',
  'COCO_train2014_000000000110.jpg',
  'COCO_train2014_000000000113.jpg',
  'COCO_train2014_000000000127.jpg',
  'COCO_train2014_000000000138.jpg',
  'COCO_train2014_000000000142.jpg',
  'COCO_train2014_000000000

In [339]:
scores = []

for i in xrange(len(titles)):
    caption_idx = np.random.randint(len(im2idx[i]))
    caption = captions[caption_idx][0]
    scores.append(n_relevant(caption, titles[i], descriptions[i], tags[i], model))
    if i % 1000 == 0:
        print i


0
1000
2000
3000
4000
5000
6000


KeyboardInterrupt: 

In [77]:
from nltk.stem import WordNetLemmatizer, LancasterStemmer, SnowballStemmer
from nltk.corpus import wordnet

wnl = WordNetLemmatizer()
lst = LancasterStemmer()
sst = SnowballStemmer("english")
print wnl.lemmatize('tables')
print lst.stem('preparing')
model.similarity('kitchen', 'cake')

a = wordnet.synsets('dessert')[0]
b = wordnet.synsets('cakes')[0]
c = wordnet.synset('prepare.v.03')
d = wordnet.synset('frosting.n.01')
print b.path_similarity(a), c.path_similarity(a), d.path_similarity(b)


table
prep
0.0833333333333 0.0909090909091 0.142857142857


In [137]:
summed_scores = []
for s in scores:
    if len(s) > 0:
        summed_scores.append(sum(s) / len(s))

import matplotlib.pyplot as plt
plt.hist(summed_scores)
plt.show()

In [131]:
scores_mismatch = []
for i in xrange(5000):
    choice_idxs = range(len(titles))
    del choice_idxs[i] # remove current
    caption_idx = np.random.randint(len(im2idx[np.random.choice(choice_idxs)]))
    caption = captions[caption_idx][0]
    scores_mismatch.append(n_relevant(caption, titles[i], descriptions[i], tags[i], model))
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000


In [138]:
summed_scores_mismatch = []
for s in scores_mismatch:
    if len(s) > 0:
        summed_scores_mismatch.append(sum(s) / len(s))

import matplotlib.pyplot as plt
plt.hist(summed_scores_mismatch)
plt.hist(summed_scores)
plt.show()

In [139]:
plt.scatter(range(len(summed_scores)), summed_scores)
plt.scatter(range(len(summed_scores_mismatch)), summed_scores_mismatch)
plt.show()

In [202]:
# context size
sizes = []
for i in xrange(len(titles)):
    count = 0
    if titles[i]:
        count += len(titles[i].split(' '))
    if descriptions[i]:
        for s in descriptions[i].split('.'):
            for w in s.split(" "):
                count += 1
    if tags[i]:
        count += len(tags[i])
    sizes.append(count)

In [204]:
plt.hist(sizes, bins=100, range=(0, 100), cumulative=False)
plt.show()

5


In [217]:
sum(sizes)

2359908

#Metric evaluation

In [153]:
import sys
import re
from collections import OrderedDict
sys.path.append('../server/pycocoevalcap/')

In [124]:
# Load captions
import cPickle
with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/coco_align.train.pkl') as f:
    cap = cPickle.load(f)

In [145]:
# Load context
with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/coco_train_context.pkl') as f:
    titles = cPickle.load(f)
    descs = cPickle.load(f)
    tags = cPickle.load(f)

[u'a restaurant has modern wooden tables and chairs .',
 u'a long restaurant table with rattan rounded back chairs .',
 u'a long table with a plant on top of it surrounded with wooden chairs',
 u'a long table with a flower arrangement in the middle for meetings',
 u'a table is adorned with wooden chairs with blue accents .']

In [156]:
# Load train list
featdict = OrderedDict()
idx = 0
with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/train2014list.txt', 'r') as f: #    ls train > train2014list.txt
    for line in f:
        line = re.sub(r'\/.*\/','',line).strip()
        featdict[line] = idx
        idx += 1
# Load split
split = []
with open('/media/Data/flipvanrijn/datasets/coco/processed/reduced/splits/coco_train.txt', 'r') as f:
    for idx, line in enumerate(f):
        split.append(line.strip())

In [142]:
import itertools
import operator
it = itertools.groupby(cap, operator.itemgetter(1))
grouped_caps = {}
for k, subit in it:
    grouped_caps[k] = [i[0] for i in subit]

In [162]:
ordered_context = [None]*len(split)
for idx, fname in enumerate(split[:10]):
    print titles[featdict[fname]], descs[featdict[fname]], tags[featdict[fname]]

long dining table annie smithers bistrot we started with the cheese puffs , and it was like biting into little puffs of air . it was so light that spearing it with a fork was too hard , and we had to resort to fingers . we suspect that it is a basic souffle mix dollopped into hot oil and fried to a light and airy puff . amazing . once i saw that the cheese puffs were a manageable size , i immediately added the duck neck sausage to our order . not long after , 4 small discs of minced pork appeared . i love a good sausage and this did disappoint . it was served with segments of orange that provided a nice foil . there were also a few boiled baby beets hidden under the leaves and it seemed a bit too tender compared to the nice big flavoursome beets we were used to . julia was very impressed with the duck fillets , and she remarked several times how tasty the flesh was . i prefered the smoky grilled quail , but it was true , the quail meat was tender and juicy like the duck , but did have 

In [163]:
import numpy as np
loader = np.load('/media/Data/flipvanrijn/datasets/coco/processed/reduced/context_train_filtered_stemmed.npz')

In [167]:
data = loader['data']

In [399]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import wordpunct_tokenize, sent_tokenize
from PIL import Image

from rouge.rouge import Rouge
e = Rouge()

stemmer = SnowballStemmer('english')

def stem(cap):
    cap = cap.split(' ')
    stemmed_cap = [stemmer.stem(w) for w in cap]
    return ' '.join(stemmed_cap)

idx = 1

image_path = '/media/Data/flipvanrijn/datasets/coco/images/train/'+split[idx]
img = Image.open(image_path)

c = '''a close up of a person eating food
a couple of people that are in a kitchen
a close up of a person cutting a cake
a couple of people that are in a kitchen .
a person sitting at a table with a plate of food
a close up of a person holding a plate of food
a couple of people sitting at a table with food .
a man sitting at a table with a plate of food
a man sitting at a table with a plate of food .
a person sitting at a table with a plate of food .
a man sitting at a table with a bowl of food .
a man sitting at a table with a cake and candles .
a man sitting at a table with a tray of food .
a couple of people sitting at a table with food on it .
a man sitting at a table in front of a plate of food .
a person sitting at a table in front of a plate of food .
a couple of people sitting at a table with food in front of them
a person sitting at a table with a plate of food and drinks .
a couple of people sitting at a table with food in front of them .
a person sitting at a table with a plate of food in front of him
a man sitting at a table with a plate of food in front of him .
a person sitting at a table with a plate of food in front of him .
a man sitting at a table with a plate of food in front of it .
a person sitting at a table with a plate of food in front of it .
a man sitting at a table with a plate of food in front of her .'''.split('\n')
cc = ' '.join(data[featdict[split[idx]]]) # cap context

refs = {0: c[15:16]}
hyp = {0: [cc]}

print e.compute_score(refs, hyp)
print c[16]

(0.065151052792187983, array([ 0.06515105]))
a couple of people sitting at a table with food in front of them


In [382]:
server_data = {
    'pixels': img.tobytes(), 
    'size': img.size, 
    'mode': img.mode, 
    'introspect': False, 
    'file_path': '/home/flipvanrijn/',
    'k': 25,
    #'text_context': np.array(data_ctx[index].todense().reshape((100, 512))),
}
server_data = cPickle.dumps(server_data)

In [383]:
import socket, struct
HOST, PORT = "localhost", 9999
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    # Connect to server and send data
    sock.connect((HOST, PORT))
    server_data = struct.pack('>I', len(server_data)) + server_data
    sock.sendall(server_data)

    # Receive data from the server and shut down
    received = sock.recv(1024)
    print received
finally:
    sock.close()

a man sitting at a table with a plate of food in front of her .


In [422]:
import nltk
nltk.pos_tag(['table'])

[('table', 'JJ')]

In [408]:
l = np.load('../dataset/coco/out.npz')
nouns = l['nouns']

In [419]:
print descs[featdict[split[0]]]
nouns[featdict[split[0]]]

we started with the cheese puffs , and it was like biting into little puffs of air . it was so light that spearing it with a fork was too hard , and we had to resort to fingers . we suspect that it is a basic souffle mix dollopped into hot oil and fried to a light and airy puff . amazing . once i saw that the cheese puffs were a manageable size , i immediately added the duck neck sausage to our order . not long after , 4 small discs of minced pork appeared . i love a good sausage and this did disappoint . it was served with segments of orange that provided a nice foil . there were also a few boiled baby beets hidden under the leaves and it seemed a bit too tender compared to the nice big flavoursome beets we were used to . julia was very impressed with the duck fillets , and she remarked several times how tasty the flesh was . i prefered the smoky grilled quail , but it was true , the quail meat was tender and juicy like the duck , but did have the richness of flavour of the duck . the

[u'annie',
 u'smithers',
 u'puffs',
 u'biting',
 u'puffs',
 u'air',
 u'fork',
 u'fingers',
 u'souffle',
 u'mix',
 u'oil',
 u'light',
 u'airy',
 u'puff',
 u'puffs',
 u'size',
 u'duck',
 u'neck',
 u'sausage',
 u'order',
 u'discs',
 u'pork',
 u'sausage',
 u'disappoint',
 u'segments',
 u'foil',
 u'baby',
 u'beets',
 u'leaves',
 u'bit',
 u'flavoursome',
 u'beets',
 u'julia',
 u'duck',
 u'fillets',
 u'times',
 u'tasty',
 u'flesh',
 u'smoky',
 u'quail',
 u'quail',
 u'meat',
 u'juicy',
 u'duck',
 u'richness',
 u'duck',
 u'pommes',
 u'quail',
 u'madeira',
 u'sauce',
 u'crispy',
 u'potato',
 u'edges',
 u'beans',
 u'tender',
 u'spring',
 u'sweetness',
 u'meal',
 u'coffee',
 u'tea',
 u'profiteroles',
 u'chocolate',
 u'sauce',
 u'choux',
 u'pastry',
 u'laurent',
 u'pattiserie',
 u'annie',
 u'smither',
 u'bistrot',
 u'produce',
 u'piper',
 u'st',
 u'kyneton',
 u'vic',
 u'reviews',
 u'smithers',
 u'necia',
 u'age',
 u'epicure',
 u'september',
 u'score',
 u'annie',
 u'smithers',
 u'age',
 u'good',
 u'